In [25]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [26]:
url = "https://www.ambitionbox.com/list-of-companies?page=1"
headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:133.0) Gecko/20100101 Firefox/133.0"}
webpage = requests.get(url, headers = headers).text

In [27]:
#!pip install lxml

In [30]:
soup=BeautifulSoup(webpage,'html.parser')

In [135]:
#print(soup.prettify())

In [45]:
companies = soup.find_all('div', class_ = "companyCardWrapper__companyDetails") 

In [46]:
len(companies)

20

In [173]:
name = []
rating = []
number_of_rating = []
services = []
headquarter = []
highly_rated = []
critically_rated = []

companies = soup.find_all('div', class_ = "companyCardWrapper__companyDetails") 

for company in companies:
    name.append(company.find("h2").text.strip())
    rating.append(company.find("div", class_="rating_text").text.strip())
    number_of_rating.append(company.find("span", class_="companyCardWrapper__companyRatingCount").text.strip().replace("(", "").replace(")", ""))
    ser_hq = company.find("span", class_="companyCardWrapper__interLinking").text.strip()
    if ser_hq:
        parts = ser_hq.split('|')
        if len(parts) >= 2:
            services.append(parts[0].strip())
            headquarter.append(parts[1].strip().split()[0])
        else:
            headquarter.append(parts[0].strip())  # First part: IT Services & Consulting
            services.append(None)

        rating_comparison = company.find("div", class_="companyCardWrapper__ratingComparisonWrapper")

    if rating_comparison:
        # Extract the "Highly Rated For" span text
        highly_rated_div = rating_comparison.find("span", class_="companyCardWrapper__ratingHeader--high")
        highly_rated_span = highly_rated_div.find_next("span", class_="companyCardWrapper__ratingValues") if highly_rated_div else None
        highly_rated.append(highly_rated_span.text.strip().replace("/", "").replace("  ", " ") if highly_rated_span else None)
    
        # Extract the "Critically Rated For" span text
        critically_rated_div = rating_comparison.find("span", class_="companyCardWrapper__ratingHeader--critical")
        critically_rated_span = critically_rated_div.find_next("span", class_="companyCardWrapper__ratingValues") if critically_rated_div else None
        critically_rated.append(critically_rated_span.text.strip().replace("/", "").replace("  ", " ") if critically_rated_span else None)


d = {"name":name,"rating":rating,"number_of_ratings":number_of_rating,"services":services,"headquarter":headquarter,"highly_rated":highly_rated,"critically_rated":critically_rated}
df = pd.DataFrame(d)       


In [174]:
df

,name,rating,number_of_ratings,services,headquarter,highly_rated,critically_rated
0,TCS,3.7,84.6k,IT Services & Consulting,Mumbai,"Job Security, Work Life Balance","Promotions Appraisal, Salary & Benefits, Work ..."
1,Accenture,3.9,52.8k,IT Services & Consulting,Dublin,"Company Culture, Job Security, Skill Developme...",Promotions Appraisal
2,Wipro,3.7,50.3k,IT Services & Consulting,Bangalore/Bengaluru,Job Security,"Promotions Appraisal, Salary & Benefits"
3,Cognizant,3.8,47.3k,IT Services & Consulting,Teaneck.,None,"Promotions Appraisal, Salary & Benefits"
4,Capgemini,3.8,39k,IT Services & Consulting,Paris,"Job Security, Work Life Balance","Promotions Appraisal, Salary & Benefits"
5,HDFC Bank,3.9,37.6k,Banking,Mumbai,"Job Security, Skill Development Learning",Promotions Appraisal
6,ICICI Bank,4.0,36.7k,Banking,Mumbai,"Job Security, Skill Development Learning, Comp...",None
7,Infosys,3.7,36.6k,IT Services & Consulting,Bengaluru/Bangalore,"Job Security, Company Culture","Promotions Appraisal, Salary & Benefits, Work ..."
8,HCLTech,3.5,33.8k,IT Services & Consulting,Noida,Job Security,"Promotions Appraisal, Salary & Benefits, Work ..."
9,Tech Mahindra,3.6,33.2k,IT Services & Consulting,Pune,None,"Promotions Appraisal, Salary & Benefits, Work ..."


In [177]:
# Initialize lists
name = []
rating = []
number_of_rating = []
services = []
headquarter = []
highly_rated = []
critically_rated = []

# Loop through pages
for page in range(1, 8):  # Adjust the range based on the total number of pages
    url = f"https://www.ambitionbox.com/list-of-companies?page={page}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:133.0) Gecko/20100101 Firefox/133.0"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract company details
    companies = soup.find_all('div', class_="companyCardWrapper__companyDetails")

    if not companies:
        print(f"No companies found on page {page}. Stopping.")
        break

    for company in companies:
        
        name_tag = company.find("h2")
        name.append(name_tag.text.strip() if name_tag else None)
        
        rating_tag = company.find("div", class_="rating_text")
        rating.append(rating_tag.text.strip() if rating_tag else None)
        
        rating_count_tag = company.find("span", class_="companyCardWrapper__companyRatingCount")
        number_of_rating.append(
            rating_count_tag.text.strip().replace("(", "").replace(")", "") if rating_count_tag else None
        )


        ser_hq_tag = company.find("span", class_="companyCardWrapper__interLinking")
        if ser_hq_tag:
            ser_hq = ser_hq_tag.text.strip()
            parts = ser_hq.split('|')
            if len(parts) >= 2:
                services.append(parts[0].strip())
                headquarter.append(parts[1].strip().split()[0])
            else:
                services.append(parts[0].strip())
                headquarter.append(None)
        else:
            services.append(None)
            headquarter.append(None)
        

        rating_comparison = company.find("div", class_="companyCardWrapper__ratingComparisonWrapper")

        if rating_comparison:
            # Extract the "Highly Rated For" span text
            highly_rated_div = rating_comparison.find("span", class_="companyCardWrapper__ratingHeader--high")
            highly_rated_span = highly_rated_div.find_next("span", class_="companyCardWrapper__ratingValues") if highly_rated_div else None
            highly_rated.append(
                highly_rated_span.text.strip().replace("/", "").replace("  ", " ") if highly_rated_span else None
            )

            # Extract the "Critically Rated For" span text
            critically_rated_div = rating_comparison.find("span", class_="companyCardWrapper__ratingHeader--critical")
            critically_rated_span = critically_rated_div.find_next("span", class_="companyCardWrapper__ratingValues") if critically_rated_div else None
            critically_rated.append(
                critically_rated_span.text.strip().replace("/", "").replace("  ", " ") if critically_rated_span else None
            )
        else:
            highly_rated.append(None)
            critically_rated.append(None)

# Create a DataFrame
d = {
    "name": name,
    "rating": rating,
    "number_of_ratings": number_of_rating,
    "services": services,
    "headquarter": headquarter,
    "highly_rated": highly_rated,
    "critically_rated": critically_rated,
}

df = pd.DataFrame(d)

# Save to a CSV file
df.to_csv("output/top_indian_companies.csv", index=False)
print("Data successfully saved'")


Data successfully saved'


In [178]:
df

,name,rating,number_of_ratings,services,headquarter,highly_rated,critically_rated
0,TCS,3.7,84.6k,IT Services & Consulting,Mumbai,"Job Security, Work Life Balance","Promotions Appraisal, Salary & Benefits, Work ..."
1,Accenture,3.9,52.9k,IT Services & Consulting,Dublin,"Company Culture, Job Security, Skill Developme...",Promotions Appraisal
2,Wipro,3.7,50.3k,IT Services & Consulting,Bangalore/Bengaluru,Job Security,"Promotions Appraisal, Salary & Benefits"
3,Cognizant,3.8,47.3k,IT Services & Consulting,Teaneck.,None,"Promotions Appraisal, Salary & Benefits"
4,Capgemini,3.8,39.1k,IT Services & Consulting,Paris,"Job Security, Work Life Balance","Promotions Appraisal, Salary & Benefits"
...,...,...,...,...,...,...,...
135,Hewlett Packard Enterprise,4.2,3.7k,IT Services & Consulting,Houston,"Company Culture, Work Life Balance, Job Security",Promotions Appraisal
136,State Bank of India,3.8,3.7k,Banking,Mumbai,"Job Security, Salary & Benefits",Work Life Balance
137,CBRE,4.2,3.7k,Real Estate,Los,"Skill Development Learning, Job Security, Comp...",None
138,Bosch,4.2,3.7k,Auto Components,Gerlingen,"Company Culture, Work Life Balance, Job Security",None
